In [2]:
import __init__

In [33]:
import numpy as np
import re
from joblib import dump, load
import pandas as pd
from stocknews_api import StockNewsAPI
from scrapy import Scrapy

In [5]:
token = load("./stocknewsapi/stocknewsapi_key.pkl")

In [6]:
start_date = "2021-06-01"
end_date = "2021-06-07"
tickers = "AAPL"

In [10]:
stocknewsapi = StockNewsAPI(token, start_date, end_date)

In [13]:
news = stocknewsapi.get_news(tickers, items=50, pages=1)

In [15]:
df_news = pd.DataFrame(news)

In [16]:
df_news.head(3)

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers
0,https://www.cnbc.com/2021/06/07/apple-is-turni...,https://cdn.snapi.dev/images/v1/0/i/106893386-...,Apple is turning privacy into a business advan...,Privacy was mentioned as part of nearly every ...,CNBC,"Mon, 07 Jun 2021 18:52:59 -0400",[],Positive,Article,[AAPL]
1,https://www.reuters.com/technology/top-takeawa...,https://cdn.snapi.dev/images/v1/h/w/m02d202106...,Factbox: Top takeaways from Apple's Developers...,Apple Inc on Monday announced a host of new fe...,Reuters,"Mon, 07 Jun 2021 16:55:00 -0400",[],Positive,Article,[AAPL]
2,https://www.cnn.com/2021/06/07/tech/wwdc-apple...,https://cdn.snapi.dev/images/v1/i/p/2106070841...,Apple's WWDC 2021 event is about to kick off,Apple is about to reveal its latest set of cha...,CNN Business,"Mon, 07 Jun 2021 12:35:37 -0400",[product],Positive,Article,[AAPL]


In [17]:
df_news.to_csv('./data/stocknews_01jun21-07jun21.csv', index=False)

In [19]:
scrap = Scrapy()

In [20]:
articles = scrap.scrap(df_news['news_url'].to_list())

In [21]:
print('Total scraped articles:', len(articles))

Total scraped articles: 50


In [25]:
np.all(df_news.shape[0] == len(articles))

True

In [26]:
dump(articles, './data/stocknewsapi_articles_01jun21-07jun21')

['./data/stocknewsapi_articles_01jun21-07jun21']

In [30]:
# prepare our data concatenating the 3 main dataframe for scrubbing
df_news = pd.concat([df_news, pd.Series(articles, name='articles')],
                   axis=1,
                   ignore_index=False,
                   sort=True)

In [31]:
df_news.head()

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,articles
0,https://www.cnbc.com/2021/06/07/apple-is-turni...,https://cdn.snapi.dev/images/v1/0/i/106893386-...,Apple is turning privacy into a business advan...,Privacy was mentioned as part of nearly every ...,CNBC,"Mon, 07 Jun 2021 18:52:59 -0400",[],Positive,Article,[AAPL],In this article Apple unveiled new versions o...
1,https://www.reuters.com/technology/top-takeawa...,https://cdn.snapi.dev/images/v1/h/w/m02d202106...,Factbox: Top takeaways from Apple's Developers...,Apple Inc on Monday announced a host of new fe...,Reuters,"Mon, 07 Jun 2021 16:55:00 -0400",[],Positive,Article,[AAPL],Apple Inc (AAPL.O) on Monday announced a host ...
2,https://www.cnn.com/2021/06/07/tech/wwdc-apple...,https://cdn.snapi.dev/images/v1/i/p/2106070841...,Apple's WWDC 2021 event is about to kick off,Apple is about to reveal its latest set of cha...,CNN Business,"Mon, 07 Jun 2021 12:35:37 -0400",[product],Positive,Article,[AAPL],"By Clare Duffy, CNN Business Updated 4:18 PM ..."
3,https://techcrunch.com/2021/06/07/apple-releas...,https://cdn.snapi.dev/images/v1/0/p/0ppldd-858...,"Apple releases torrent of updates, and Wall St...",Today's WWDC keynote from Apple covered a huge...,TechCrunch,"Mon, 07 Jun 2021 15:55:00 -0400",[],Neutral,Article,[AAPL],NaN
4,https://www.benzinga.com/news/21/06/21457495/a...,https://cdn.snapi.dev/images/v1/c/o/computer-e...,Apple's Introduces FaceTime Calls with Multipl...,Apple Inc (NASDAQ: AAPL) introduced updates to...,Benzinga,"Mon, 07 Jun 2021 14:02:01 -0400",[product],Positive,Article,[AAPL],© 2021 Benzinga.com. Benzinga does not provide...


In [32]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   news_url     50 non-null     object
 1   image_url    50 non-null     object
 2   title        50 non-null     object
 3   text         50 non-null     object
 4   source_name  50 non-null     object
 5   date         50 non-null     object
 6   topics       50 non-null     object
 7   sentiment    50 non-null     object
 8   type         50 non-null     object
 9   tickers      50 non-null     object
 10  articles     44 non-null     object
dtypes: object(11)
memory usage: 4.7+ KB


In [40]:
df_news.rename(columns={'source_name':'source'}, inplace=True)

In [34]:
df_news['date'].str.findall("\s-[0-9]{4}$").apply(lambda x: ''.join(x)).unique()

array([' -0400'], dtype=object)

In [35]:
df_news['date'] = df_news['date'].str.replace("\s-[0-9]{4}$", '', regex=True)

In [37]:
df_news['date'] = pd.to_datetime(df_news['date'], format="%a, %d %b %Y %H:%M:%S")

In [38]:
df_news.loc[:5,'date']

0   2021-06-07 18:52:59
1   2021-06-07 16:55:00
2   2021-06-07 12:35:37
3   2021-06-07 15:55:00
4   2021-06-07 14:02:01
5   2021-06-07 13:20:23
Name: date, dtype: datetime64[ns]

In [42]:
df_news.head()

,news_url,image_url,title,text,source,date,topics,sentiment,type,tickers,articles
0,https://www.cnbc.com/2021/06/07/apple-is-turni...,https://cdn.snapi.dev/images/v1/0/i/106893386-...,Apple is turning privacy into a business advan...,Privacy was mentioned as part of nearly every ...,CNBC,2021-06-07 18:52:59,[],Positive,Article,[AAPL],In this article Apple unveiled new versions o...
1,https://www.reuters.com/technology/top-takeawa...,https://cdn.snapi.dev/images/v1/h/w/m02d202106...,Factbox: Top takeaways from Apple's Developers...,Apple Inc on Monday announced a host of new fe...,Reuters,2021-06-07 16:55:00,[],Positive,Article,[AAPL],Apple Inc (AAPL.O) on Monday announced a host ...
2,https://www.cnn.com/2021/06/07/tech/wwdc-apple...,https://cdn.snapi.dev/images/v1/i/p/2106070841...,Apple's WWDC 2021 event is about to kick off,Apple is about to reveal its latest set of cha...,CNN Business,2021-06-07 12:35:37,[product],Positive,Article,[AAPL],"By Clare Duffy, CNN Business Updated 4:18 PM ..."
3,https://techcrunch.com/2021/06/07/apple-releas...,https://cdn.snapi.dev/images/v1/0/p/0ppldd-858...,"Apple releases torrent of updates, and Wall St...",Today's WWDC keynote from Apple covered a huge...,TechCrunch,2021-06-07 15:55:00,[],Neutral,Article,[AAPL],NaN
4,https://www.benzinga.com/news/21/06/21457495/a...,https://cdn.snapi.dev/images/v1/c/o/computer-e...,Apple's Introduces FaceTime Calls with Multipl...,Apple Inc (NASDAQ: AAPL) introduced updates to...,Benzinga,2021-06-07 14:02:01,[product],Positive,Article,[AAPL],© 2021 Benzinga.com. Benzinga does not provide...
